## Callin Switzer
## Run Simulations to compare output to NNET

In [6]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import seaborn as sns
from scipy.integrate import odeint
import random
import time
from datetime import datetime
import sys
import pandas as pd
import multiProcTraj # note that this is a custom-written file 


# define directories
baseDir = os.getcwd()
dataDir = r'D:\MothSimulations\11c-AggressiveManeuver\Qstore\hws_am_con'
figDir = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\Figs'
dataOutput = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\DataOutput'
savedModels = r'D:\Dropbox\AcademiaDropbox\mothMachineLearning_dataAndFigs\savedModels'
randomRawData = r'D:/Dropbox/AcademiaDropbox/mothMachineLearning_dataAndFigs/PythonGeneratedData'

if not os.path.exists(dataOutput):
    os.mkdir(dataOutput)
if not os.path.exists(savedModels):
    os.mkdir(savedModels)

print(sys.version)
now = datetime.now()
print("last run on " + str(now))

3.6.6 | packaged by conda-forge | (default, Jul 26 2018, 11:48:23) [MSC v.1900 64 bit (AMD64)]
last run on 2019-01-01 09:09:32.414112


In [7]:
# Load data
testDF = pd.read_csv(os.path.join(dataOutput, "NNpreds_RandomICs.csv"))
testDF_sm= pd.read_csv(os.path.join(dataOutput, "NNpreds_small_RandomICs.csv"))
testDF_sm.head()

,x_0,y_0,phi_0,theta_0,x_dot_0,y_dot_0,phi_dot_0,theta_dot_0,F_pred,alpha_pred,tau_pred,x_99,y_99,phi_99,theta_99,x_dot_99_pred,y_dot_99_pred,phi_dot_99_pred,theta_dot_99_pred
0,0.0,0.0,5.717836,2.903536,1140.923218,-399.095208,-20.919668,7.162696,5242.905,0.227884,87642.836,21.425903,-8.116010,5.438490,2.756136,966.60364,-434.46356,-11.772325,-5.640531
1,0.0,0.0,3.799657,4.295602,1216.867455,1286.049934,9.030914,5.734873,21598.404,2.597742,-67889.390,27.849758,27.358930,3.870090,4.277221,1548.62660,1466.32930,-1.445739,-6.679240
2,0.0,0.0,1.426920,3.170749,1008.157261,175.219341,-19.672703,-17.241171,23363.543,1.863295,-15468.784,20.439296,-0.898106,1.004533,2.738070,1000.94440,-266.57200,-23.733850,-25.134838
3,0.0,0.0,1.401717,4.076364,-1022.919335,-980.612881,-9.282598,-20.555771,32194.880,3.678808,68082.640,-19.039632,-13.359467,1.199188,3.981795,-955.95240,-369.71906,-4.945975,0.057465
4,0.0,0.0,0.350069,5.063611,827.387703,1371.685241,2.420546,-2.941864,15569.313,3.173099,-8812.373,15.881660,29.669649,0.238542,4.952067,745.15857,1623.83780,-10.901551,-11.416786


In [25]:
# specify ranges
# x,xd,y,yd,
# theta,thetad,phi,phid, 
# F, alpha, tau0
ranges = np.array([[0, 0], [-1500, 1500], [0, 0], [-1500, 1500],   
                   [0, 2*np.pi], [-25, 25], [0, 2*np.pi], [-25, 25], 
                  [0, 44300], [0, 2*np.pi], [-100000, 100000]])

tic = time.time()
t = np.linspace(0, 0.02, num = 100, endpoint = False) # time cut into 100 timesteps
nrun = 100  #number of trajectories.

# initialize the matrix of 0's
zeroMatrix = np.zeros([nstep, nrun])
x,      xd,    y,   yd, \
theta, thetad, phi, phid = [zeroMatrix.copy() for ii in 
                                range(len([ "x",     "xd",     "y", "yd", 
                                            "theta", "thetad", "phi", "phid"]))]

# generate random initial conditions for state 0
state0 = np.random.uniform(ranges[:, 0], ranges[:, 1], 
                           size=(nrun, ranges.shape[0]))

# loop through all the runs
for i in range(0,nrun):
    # run ODE
    state = odeint(FlyTheBug, state0[i, :], t)
    x[i,:], xd[i,:] = state[:,0], state[:,1]
    y[i,:], yd[i, :] = state[:,2], state[:,3]
    theta[i,:], thetad[i, :] = state[:,4],state[:,5]
    phi[i,:], phid[i, :] = state[:, 6], state[:,7]
print('elapsed time = ',time.time()-tic)

elapsed time =  2.109321355819702
